## Mer Diagrams

### 0.0. Requirements

#### 0.1. Imports

In [1]:
import os
import re
import graphviz

from eralchemy import render_er
from sqlalchemy.engine import create_engine
from IPython.display import HTML, display, Image

from warnings import filterwarnings

filterwarnings('ignore')

In [2]:
from sys import platform
from sqlalchemy import __version__

print(platform)
print(f'SQLAlchemy Version: {__version__}')

linux
SQLAlchemy Version: 1.4.47


### 1.0. Mer Diagrams

#### 1.1. Load Data

In [ ]:
ssms_db   = os.environ.get("ssms_db")
ssms_host = os.environ.get("ssms_host")
ssms_user = os.environ.get("ssms_user")
ssms_pswd = os.environ.get("ssms_pswd")

In [4]:
string_con_sql_server = f"mssql+pyodbc:///?odbc_connect=DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={ssms_host};DATABASE={ssms_db};UID={ssms_user};PWD={ssms_pswd}"

db = create_engine(string_con_sql_server)
con = db.connect()

In [7]:
dw_tables = con.execute(f"SELECT TABLE_SCHEMA, TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME like 'D[_]%' ORDER BY 1")
dims = dw_tables.fetchall()

# Remove Test and Bkp Tables 
dims = [dim_table for dim_table in dims if not (re.findall('bkp', dim_table[-1].lower()) or re.findall('test', dim_table[-1].lower()))]

In [8]:
dw_tables = con.execute(f"SELECT TABLE_SCHEMA, TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME like 'F[_]%' ORDER BY 1")
facts = dw_tables.fetchall()

# Remove Test and Bkp Tables 
facts = [fac_table for fac_table in facts if not (re.findall('bkp', fac_table[-1].lower()) or re.findall('test', fac_table[-1].lower()))]

In [9]:
SCHEMAS = list(set([k[0] for k in facts]))

dims_name_list = [k[-1] for k in dims]
#dims_and_facts = list(set([k[-1] for k in dims])) + list(set([k[-1] for k in facts]))

In [11]:
SCHEMAS, dims_name_list

(['DMOls'], ['D_CLIENTE', 'D_PRODUTO', 'D_VENDEDOR'])

#### 1.2. Get Facts -> Dims

In [ ]:
dms_dict = {}

for dm in SCHEMAS:
    print("\n" + dm + "\n")

    dms_dict[dm] = {}
    dm_facts_list = con.execute(f"SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{dm}' AND TABLE_NAME like 'F_%' ORDER BY 1")

    for fact in [k[0] for k in dm_facts_list.fetchall()]: # remove fetchall tuples
        if re.findall('bkp', fact.lower()) or re.findall('test', fact.lower()): 
            del fact
            continue

        fact_cols_query = f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = '{dm}' AND TABLE_NAME = '{fact}'"
        fact_cols = con.execute(fact_cols_query).fetchall()
        fact_cols = [k[0] for k in fact_cols]

        fact_join_dims = [ # Get all Dims Cols for One Fact
            k.replace('SK_F_', 'F_').replace('SK_D_', 'D_').replace('SK_', 'D_') for k in fact_cols \
                if (re.search('SK_D_', k) or re.search('D_', k) or re.search('SK_', k))
        ]

        dims_exists = []
        for d in fact_join_dims:
            if d not in dims_name_list:
                print(f"\t{dm}.{d} Not Exists")
            else: 
                dims_exists.append(d)

        #dims_exists = [d for d in fact_join_dims if d in dims_name_list]

        # Remove Facts if exists on dims list
        dims_exists = [k for k in dims_exists if not k.startswith('F_')]
        
        dms_dict[dm][fact] = {'cols': fact_cols, 'dims': dims_exists}

In [13]:
dms_dict[dm]

{'F_ORDEM_ITEM': {'cols': ['SK_F_ORDEM_ITEM',
   'SK_D_CLIENTE',
   'SK_D_VENDEDOR',
   'SK_D_PRODUTO',
   'CODIGO_ORDEM',
   'STATUS',
   'TIPO_PAGAMENTO',
   'DATA_ORDEM_COMPRA',
   'DATA_ORDEM_APROVACAO',
   'DATA_DESPACHO',
   'DATA_ENTREGA_CLIENTE',
   'DATA_ESTIMATIVA_ENTREGUA',
   'TOTAL_PRECO_ITEM',
   'TOTAL_PRECO_FRETE',
   'QUANTIDADE_PARCELAS',
   'QUANTIDADE_INSTALLMENTS',
   'TOTAL_PRECO_PAGAMENTO',
   'DATA_PROCESSAMENTO'],
  'dims': ['D_CLIENTE', 'D_VENDEDOR', 'D_PRODUTO']}}

#### 1.3. Generate Fact Mer Diagram

In [17]:
for dm in SCHEMAS:
    print(dm)
    if not os.path.isdir(f'pngs/{dm}'): os.makedirs(f"pngs/{dm}")
    if not os.path.isdir(f'dots/{dm}'): os.makedirs(f"dots/{dm}")

    for fact in dms_dict[dm].keys():
        print("\t"+fact)
        
        file_path = f'dots/{dm}/{dm}_{fact}.dot'
        img_path = f'pngs/{dm}/{dm}_{fact}'

        # Check if have dims join on fact *
        if dms_dict[dm][fact]['dims']:
            include_tables = f"{dm}.{fact} {dm}." + f" {dm}.".join(dms_dict[dm][fact]['dims'])
        
        else:
            include_tables = f"{dm}.{fact}"

        render_er(string_con_sql_server, file_path, schema=dm, include_tables=include_tables)

        with open(file_path) as dot:
            dot_ = dot.read()

        size = round(len(dms_dict[dm][fact]['dims'])/2)
        ext_lines = f';\n"' 
        ext_lines += f'"--"{fact}";\n"'.join(dms_dict[dm][fact]['dims'][:size]) 
        ext_lines += f'"--"{fact}";\n"{fact}"--"'
        ext_lines += f'";\n"{fact}"--"'.join(dms_dict[dm][fact]['dims'][size:]) 
        ext_lines += '";\n\n}'
        
        dot_string = dot_.replace('digraph', 'graph'). \
                          replace(' -> ', ' -- ').replace(f'{dm}.', ''). \
                          replace(' COLLATE "SQL_Latin1_General_CP1_CI_AS"', '').\
                          replace(' COLLATE "SQL_Latin1_General_CP1_CI_AI"', ''). \
                          replace(";\n\n}", ext_lines)

        src = graphviz.Source(dot_string)
        src.format = 'png'
        src.render(img_path)
        os.remove(img_path)

con.close()
db.dispose()

DMOls
	F_ORDEM_ITEM
